## Import required libraries:

In [1]:
import pandas as pd

## Load the excel data:

In [18]:
ta = pd.read_excel('data/meta/Transit_Agencies_for_Visualization.xlsx', sheet_name='TC AgencyList')
upt = pd.read_excel('data/ntd/TS2.2TimeSeriesSysWideOpexpSvc_2.xlsx', sheet_name='UPT')

## Clean the data

In [19]:
# Remove missing NTD ID's
ta = ta.dropna(how='all')
upt = upt.dropna(subset=['NTD ID'])

     ShowIndividual  Project ID  "Other" primary Project ID  Primary UZA  \
0               1.0         1.0                         NaN          NaN   
1               0.0         NaN                         1.0          1.0   
2               1.0         2.0                         NaN          1.0   
3               0.0         NaN                         1.0          1.0   
4               1.0         3.0                         NaN          1.0   
5               1.0         4.0                         NaN          1.0   
6               1.0         5.0                         NaN          1.0   
7               1.0         6.0                         NaN          1.0   
8               1.0         7.0                         NaN          1.0   
9               1.0         8.0                         NaN          1.0   
10              1.0         9.0                         NaN          1.0   
11              0.0         NaN                         9.0          1.0   
12          

## Merge the data

In [20]:
# Merging dataframes
merge = pd.merge(ta, upt, how='left', on='NTD ID')
print(merge)

     ShowIndividual  Project ID  "Other" primary Project ID  Primary UZA  \
0               1.0         1.0                         NaN          NaN   
1               0.0         NaN                         1.0          1.0   
2               1.0         2.0                         NaN          1.0   
3               0.0         NaN                         1.0          1.0   
4               1.0         3.0                         NaN          1.0   
5               1.0         4.0                         NaN          1.0   
6               1.0         5.0                         NaN          1.0   
7               1.0         6.0                         NaN          1.0   
8               1.0         7.0                         NaN          1.0   
9               1.0         8.0                         NaN          1.0   
10              1.0         9.0                         NaN          1.0   
11              0.0         NaN                         9.0          1.0   
12          